In [2]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import train_test_split
import scipy.sparse as sp

plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'


def import_tweets(filename, header = None):
    #import data from csv file via pandas library
    tweet_dataset = pd.read_csv(filename,header = header, encoding='Latin-1', low_memory=False, index_col=False)
    #the column names are based on sentiment140 dataset provided on kaggle
    tweet_dataset.columns = ['target','id','date','flag','user','text']
    #delete 3 columns: flags,id,user, as they are not required for analysi
    for i in ['flag','id','user','date']: del tweet_dataset[i] # or tweet_dataset = tweet_dataset.drop(["id","user","date","user"], axis = 1)
    #in sentiment140 dataset, positive = 4, negative = 0; So we change positive to 1
    tweet_dataset.target = tweet_dataset.target.replace(4,1)
    return tweet_dataset

def preprocess_tweet(tweet):
	#Preprocess the text in a single tweet
	#arguments: tweet = a single tweet in form of string
	#convert the tweet to lower case

	str(tweet)
	tweet.lower()
	#convert all urls to sting "URL "
	tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
	#convert all @username to "AT_USER "
	tweet = re.sub('@[^\s]+','AT_USER', tweet)
	#correct all multiple white spaces to a single white space
	tweet = re.sub('[\s]+', ' ', tweet)
	#convert "#topic" to just "topic"
	tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
	return tweet




In [3]:
my_df = import_tweets("training_1600000_processed_noemoticon.csv")
print("File read")

my_df['text'] = my_df['text'].apply(preprocess_tweet)
print("Preprocessing done")

# csv = 'clean_tweet.csv'
# my_df = pd.read_csv(csv,index_col=0)
my_df.head()

File read


Preprocessing done


,target,text
0,0,"AT_USER URL - Awww, that's a bummer. You shoul..."
1,0,is upset that he can't update his Facebook by ...
2,0,AT_USER I dived many times for the ball. Manag...
3,0,my whole body feels itchy and like its on fire
4,0,"AT_USER no, it's not behaving at all. i'm mad...."


In [4]:
my_df.dropna(inplace=True)
my_df.reset_index(drop=True,inplace=True)
my_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 2 columns):
target    1600000 non-null int64
text      1600000 non-null object
dtypes: int64(1), object(1)
memory usage: 24.4+ MB


In [5]:
x = my_df.text
y = my_df.target

In [6]:
SEED = 2000
x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(x, y, test_size=.1, random_state=SEED)
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=SEED)

In [7]:
print( "Train set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(x_train),
                                                                             (len(x_train[y_train == 0]) / (len(x_train)*1.))*100,
                                                                            (len(x_train[y_train == 1]) / (len(x_train)*1.))*100))
print ("Validation set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(x_validation),
                                                                             (len(x_validation[y_validation == 0]) / (len(x_validation)*1.))*100,
                                                                            (len(x_validation[y_validation == 1]) / (len(x_validation)*1.))*100))
print ("Test set has total {0} entries with {1:.2f}% negative, {2:.2f}% positive".format(len(x_test),
                                                                             (len(x_test[y_test == 0]) / (len(x_test)*1.))*100,
                                                                            (len(x_test[y_test == 1]) / (len(x_test)*1.))*100))

Train set has total 1440000 entries with 49.97% negative, 50.03% positive
Validation set has total 80000 entries with 50.27% negative, 49.73% positive
Test set has total 80000 entries with 50.18% negative, 49.82% positive


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvec1 = TfidfVectorizer(max_features=100000,ngram_range=(1, 3))
# tvec1 = TfidfVectorizer(sublinear_tf=True,stop_words="english")  # we need to give proper stopwords list for better performance

# tvec1.fit(x_train)
x_train_tfidf = tvec1.fit_transform(x_train)
print("Done")

Done


In [19]:
print("Done1")
x_validation_tfidf = tvec1.transform(x_validation)
# x_validation_tfidf = tvec1.transform(x_validation).toarray()

print("Done2")

Done1


Done2


In [20]:
%%time
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(x_train_tfidf, y_train)

C:\Users\hp\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Wall time: 1min 6s


In [21]:
clf.score(x_validation_tfidf, y_validation)

0.8221

In [22]:
clf.score(x_train_tfidf, y_train)

0.8416604166666667

I will first start by loading required dependencies. In order to run Keras with TensorFlow backend, you need to install both TensorFlow and Keras.

In [23]:
seed = 7
np.random.seed(seed)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

The structure of below NN model has 100,000 nodes in the input layer, then 64 nodes in a hidden layer with Relu activation function applied, then finally one output layer with sigmoid activation function applied. There are different types of optimizing techniques for neural networks, and different loss function you can define with the model. Below model uses ADAM optimizing, and binary cross entropy loss.

ADAM is an optimization algorithm for updating the parameters and minimizing the cost of the neural network, which is proved to be very effective. It combines two methods of optimization: RMSProp, Momentum. Again, I will focus on sharing the result I got from my implementation, but if you want to understand properly how ADAM works, I strongly recommend the "deeplearning.ai" course by Andrew Ng. He explains the complex concept of neural network in a very intuitive way.

Before I feed the data and train the model, I need to deal with one more thing. Keras NN model cannot handle sparse matrix directly. The data has to be dense array or matrix, but transforming the whole training data Tfidf vectors of 1.5 million to dense array won't fit into my RAM. So I had to define a function, which generates iterable generator object so that it can be fed to NN model. Note that the output should be a generator class object rather than arrays, this can be achieved by using "yield" instead of "return". 

In [24]:
def batch_generator(X_data, y_data, batch_size):
    samples_per_epoch = X_data.shape[0]
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    index = np.arange(np.shape(y_data)[0])
    while 1:
        index_batch = index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X_data[index_batch,:].toarray()
        y_batch = y_data[y_data.index[index_batch]]
        counter += 1
        yield X_batch,y_batch
        if (counter > number_of_batches):
            counter=0

It looks like the model had the best validation accuracy after 2 epochs, and after that, it fails to generalize so validation accuracy slowly decreases, while training accuracy increases. But if you remember the result I got from logistic regression (train accuracy: 84.19%, validation accuracy: 82.91%), you can see that the above neural network failed to outperform logistic regression in terms of validation.

Let's see if normalizing inputs have any effect on the performance.

By the look of the result, normalizing seems to have almost no effect on the performance. And it is at this point I realized that Tfidf is already normalized by the way it is calculated. TF (Term Frequency) in Tfidf is not absolute frequency but relative frequency, and by multiplying IDF (Inverse Document Frequency) to the relative term frequency value, it further normalizes the value in a cross-document manner.

If the problem of the model is a poor generalization, then there is another thing I can add to the model. Even though the neural network is a very powerful model, sometimes overfitting to the training data can be a problem. Dropout is a technique that addresses this problem. If you are familiar with the concept of ensemble model in machine learning, dropout can also be seen in the same vein. According to the research paper "Improving neural networks by preventing
co-adaptation of feature detectors" by Hinton et al. (2012), "A good way to reduce the error on the test set is to
average the predictions produced by a very large number of different networks. The standard way to do this is to train many separate networks and then to apply each of these networks to the test data, but this is computationally expensive during both training and testing. Random dropout makes it possible to train a huge number of different networks in a reasonable time." https://arxiv.org/pdf/1207.0580.pdf

Dropout is simulating as if we train many different networks and averaging them by randomly omitting hidden nodes with a certain probability throughout the training process. With Keras, this can be easily implemented just by adding one line to your model architecture. Let's see how the model performance changes with 20% dropout rate. (*I will gather all the results and present them with a table at the end.)

In [26]:
model1 = Sequential()
model1.add(Dense(64, activation='relu', input_dim=100000))
model1.add(Dropout(0.2))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model1.fit_generator(generator=batch_generator(x_train_tfidf, y_train, 32),
                    epochs=5, validation_data=(x_validation_tfidf, y_validation),
                    steps_per_epoch=x_train_tfidf.shape[0]/32)

fileName = 'Logistic_tensorflow.pickle'
with open(fileName, 'wb') as file:
    pickle.dump(model1, file)

print("Pickle File Created %s" % fileName)


Epoch 1/5


    1/45000 [..............................] - ETA: 30:34:09 - loss: 0.6930 - acc: 0.6250

    2/45000 [..............................] - ETA: 16:54:29 - loss: 0.6933 - acc: 0.4844

    3/45000 [..............................] - ETA: 12:18:46 - loss: 0.6932 - acc: 0.4688

    4/45000 [..............................] - ETA: 10:04:39 - loss: 0.6931 - acc: 0.4922

...

  212/45000 [..............................] - ETA: 3:01:16 - loss: 0.6449 - acc: 0.6928

  213/45000 [..............................] - ETA: 3:01:15 - loss: 0.6448 - acc: 0.6928

  214/45000 [..............................] - ETA: 3:01:15 - loss: 0.6445 - acc: 0.6933

  215/45000 [..............................] - ETA: 3:01:11 - loss: 0.6442 - acc: 0.6937

  216/45000 [..............................] - ETA: 3:01:11 - loss: 0.6437 - acc: 0.6946